In [1]:
import sqlite3
import datetime
import pandas as pd
import pyodbc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
conn = sqlite3.connect('Tripsdata.db')

In [5]:
df = pd.read_sql_query('SELECT * FROM "Trips Data"', conn)

In [6]:
conn.close()

In [40]:
#df['Date_Float'] = df['Date'].apply(lambda x: float(datetime.datetime.strptime(x[:10], '%Y-%m-%d').timestamp()))
#did not end up using for all

In [7]:
df.drop(columns=["Date", "Level", "State Postal Code", "Row ID"], axis=1, inplace=True)

In [9]:
#df.drop(columns=["Date_Float"], axis=1, inplace=True)
#did not end up using for all

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45267 entries, 0 to 45266
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   State FIPS                      45267 non-null  float64
 1   Population Staying at Home      45267 non-null  float64
 2   Population Not Staying at Home  45267 non-null  float64
 3   Number of Trips                 45267 non-null  float64
 4   Number of Trips <1              45267 non-null  float64
 5   Number of Trips 1-3             45267 non-null  float64
 6   Number of Trips 3-5             45267 non-null  float64
 7   Number of Trips 5-10            45267 non-null  float64
 8   Number of Trips 10-25           45267 non-null  float64
 9   Number of Trips 25-50           45267 non-null  float64
 10  Number of Trips 50-100          45267 non-null  float64
 11  Number of Trips 100-250         45267 non-null  float64
 12  Number of Trips 250-500         

In [9]:
df.head()

,State FIPS,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500,Week,Month
0,32.0,753054.0,2272063.0,9139676.0,2999886.0,2181216.0,1039500.0,1378024.0,1108436.0,222837.0,93301.0,73578.0,24471.0,18427.0,0.0,1.0
1,12.0,4524666.0,16709556.0,55258300.0,16036792.0,14469767.0,6658624.0,7977612.0,6604304.0,2113009.0,796740.0,412224.0,95341.0,93887.0,0.0,1.0
2,11.0,241030.0,459278.0,3114055.0,1314825.0,833796.0,367547.0,344116.0,183744.0,42096.0,10730.0,8109.0,3476.0,5616.0,0.0,1.0
3,10.0,224457.0,739757.0,2420942.0,584323.0,640348.0,305441.0,370359.0,314102.0,128454.0,50542.0,22701.0,2474.0,2198.0,0.0,1.0
4,9.0,853894.0,2707851.0,9468961.0,2375021.0,2618933.0,1189575.0,1331356.0,1211252.0,449947.0,192960.0,56618.0,12031.0,31268.0,0.0,1.0


In [10]:
df3 = df.sample(frac=0.50, random_state=42)

In [11]:
df3

,State FIPS,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500,Week,Month
29784,37.0,2185271.0,8198349.0,33238726.0,7499378.0,7867283.0,4076161.0,5325751.0,5423975.0,1941941.0,676540.0,336331.0,75765.0,15601.0,31.0,8.0
30902,32.0,854560.0,2179832.0,7346664.0,1792047.0,1774855.0,888777.0,1263525.0,1198919.0,198125.0,100071.0,91498.0,25254.0,13593.0,34.0,8.0
16315,18.0,1550587.0,5120839.0,28031771.0,6848939.0,6564219.0,3491268.0,4534797.0,4684751.0,1410864.0,353449.0,101211.0,23683.0,18590.0,45.0,11.0
37225,33.0,366188.0,990270.0,4069649.0,822782.0,969234.0,509470.0,633106.0,734546.0,285422.0,93229.0,17751.0,1920.0,2189.0,52.0,12.0
27085,18.0,1528861.0,5163017.0,21190507.0,5587825.0,5353241.0,2655207.0,3167160.0,2809851.0,938568.0,419259.0,196363.0,45801.0,17232.0,24.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15488,37.0,1709519.0,8642362.0,42315728.0,9056128.0,10409591.0,5448154.0,7191495.0,7414671.0,2255604.0,399185.0,95452.0,22776.0,22672.0,43.0,10.0
22526,19.0,765404.0,2390741.0,10911986.0,2832558.0,2747392.0,1229571.0,1471735.0,1529566.0,636914.0,266845.0,161210.0,28383.0,7812.0,11.0,3.0
19892,8.0,1370109.0,4325455.0,22004026.0,5771566.0,5904896.0,2728190.0,3242457.0,2892472.0,871014.0,391019.0,138395.0,32806.0,31211.0,4.0,1.0
13639,51.0,1488398.0,7003257.0,40858208.0,9663802.0,9867699.0,5173811.0,7098349.0,6654944.0,1782115.0,395157.0,120724.0,40744.0,60863.0,38.0,9.0


In [12]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15843 entries, 29784 to 9089
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   State FIPS                      15843 non-null  float64
 1   Population Staying at Home      15843 non-null  float64
 2   Population Not Staying at Home  15843 non-null  float64
 3   Number of Trips                 15843 non-null  float64
 4   Number of Trips <1              15843 non-null  float64
 5   Number of Trips 1-3             15843 non-null  float64
 6   Number of Trips 3-5             15843 non-null  float64
 7   Number of Trips 5-10            15843 non-null  float64
 8   Number of Trips 10-25           15843 non-null  float64
 9   Number of Trips 25-50           15843 non-null  float64
 10  Number of Trips 50-100          15843 non-null  float64
 11  Number of Trips 100-250         15843 non-null  float64
 12  Number of Trips 250-500      

## Random Forest Model 1

In [49]:
dataset2=df3
X=dataset2[['State FIPS','Population Staying at Home','Population Not Staying at Home','Week','Month']]
y=dataset2[['Number of Trips']]

In [50]:
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X,y)

<ipython-input-50-c0179386adc8>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X,y)


RandomForestRegressor(n_estimators=20, random_state=0)

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

<ipython-input-51-c4a3fd6acd40>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


0.9899830032447869

In [52]:
y_pred=regressor.predict(X)

In [53]:
from sklearn.metrics import mean_squared_error as msc
from math import sqrt
rms=sqrt(msc(y,y_pred))
print('RMSE= ',rms) 

RMSE=  1448915.983114444


## Random Forest Model 2

In [31]:
dataset3=df3
X=dataset3[['State FIPS','Population Staying at Home','Population Not Staying at Home','Week','Month','Date_Float']]
y=dataset3[['Number of Trips']]

In [ ]:
#'Number of Trips <1','Number of Trips 1-3','Number of Trips 3-5','Number of Trips 5-10','Number of Trips 10-25','Number of Trips 25-50','Number of Trips 50-100','Number of Trips 100-250','Number of Trips 250-500','Number of Trips >=500'

In [32]:
regressor=RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X,y)

<ipython-input-32-478445dab820>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X,y)


RandomForestRegressor(n_estimators=20, random_state=0)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
r2_score(y_test, y_pred)

<ipython-input-33-081c411ff6ec>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


0.9936784174800054

## Multiple Linear Regression Model 1

In [32]:
X = df3[['State FIPS','Population Staying at Home','Population Not Staying at Home','Week','Month']]
y = df3['Number of Trips']

In [33]:
import statsmodels.api as sm

In [34]:
X = sm.add_constant(X)

In [35]:
model = sm.OLS(y, X).fit()

In [36]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        Number of Trips   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                 5.351e+04
Date:                Mon, 17 Apr 2023   Prob (F-statistic):               0.00
Time:                        22:41:19   Log-Likelihood:            -2.7038e+05
No. Observations:               15843   AIC:                         5.408e+05
Df Residuals:                   15837   BIC:                         5.408e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

## Multiple Linear Regression Model 2

In [13]:
X = df3[['State FIPS','Population Staying at Home','Population Not Staying at Home','Week','Month']]
y = df3['Number of Trips']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
regressor = LinearRegression()

In [16]:
regressor.fit(X_train, y_train)

LinearRegression()

In [17]:
regressor.fit(X_train, y_train)

LinearRegression()

In [18]:
y_pred = regressor.predict(X_test)

In [22]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)

In [23]:
r2

0.941802486791338